In [7]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xarray as xr
import os
import pingouin as pg
from tqdm import tqdm
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, spearmanr, zscore, wilcoxon
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
sys.path.append('..')
import circletrack_neural as ctn
import circletrack_behavior as ctb
import pca_ica as ica
import plotting_functions as pf

In [3]:
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']
## Create session_id dictionary
session_dict = {'mc03': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal4', 'Training_Reversal'],
                'mc06': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc07': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc09': ['Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc11': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']}
## Set path variables
ensemble_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/assemblies'
behavior_path = '../../EnsembleRemodeling_Resubmission/circletrack_data'  

In [58]:
## Set parameters
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09'] ## excluded mc11
x_bin_size = 1
analysis_type = 'average_value'
alpha_old = 0.05
correction = 'sidak'
cohort = 'cohort1'
figure_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/python_analysis/progress_figures/{}'.format(cohort)
## Loop through each mouse
mouse_ensemble_trends = {}
mouse_ensemble_proportions = {}
for mouse in tqdm(mouse_list):
    ## saved ensemble data path
    spath = pjoin(ensemble_path, mouse)
    session_trends = {}
    session_proportions = {}
    for session in session_dict[mouse]:
        ## Load a specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, spath = spath, session_id = session)
        act = assemblies.activations.values
        ## Calculate ensemble trends
        ensemble_trends = ica.ensemble_trends_linear_regression(act, x_bin_size = x_bin_size, analysis_type = analysis_type, alpha_old = alpha_old, correction = correction)
        session_trends[session] = ensemble_trends
        ## Calculate proportions
        session_proportions[session] = ica.calculate_proportions_ensembles(ensemble_trends, function = 'linear_regression')
    mouse_ensemble_trends[mouse] = session_trends
    mouse_ensemble_proportions[mouse] = session_proportions

100%|██████████| 4/4 [00:25<00:00,  6.28s/it]


In [61]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_ensemble_proportions:
    prop_decreasing_T4.append(mouse_ensemble_proportions[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_ensemble_proportions[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'Mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'grey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'Mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Time',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image(pjoin(figure_path, 'proportion_fading_trials.png'))

DO THE SAME ANALYSIS FOR COHORT0

In [62]:
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1']
## Set path variables
dpath = '/media/caishuman/csstorage/phild/git/MazeProjects/output/assemblies'
behav_path = '/media/caishuman/csstorage/phild/git/MazeProjects/output/behav'
## Create young and old mouse list
young_mice = ['Fornax', 'Janus', 'Lyra', 'Miranda', 'Naiad', 'Sao', 'Titania']
old_mice = ['Gemini', 'Oberon', 'Puck', 'Rhea', 'Umbriel', 'Virgo', 'Ymir', 'Atlas']
## Create list of female or male young mice
young_female_mice = ['Fornax', 'Janus', 'Lyra', 'Sao', 'Titania']
male_young_mice = ['Miranda', 'Naiad']

In [65]:
## Set parameters
x_bin_size = 1
analysis_type = 'average_value'
alpha_old = 0.05
correction = 'sidak'
cohort = 'cohort0'
figure_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/python_analysis/progress_figures/{}'.format(cohort)
## Loop through each mouse
mouse_ensemble_trends = {}
mouse_ensemble_proportions = {}
for mouse in tqdm(young_mice):
    ## saved ensemble data path
    session_trends = {}
    session_proportions = {}
    for session in session_list:
        ## Load a specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, spath = dpath, session_id = session)
        act = assemblies.activations.values
        ## Calculate ensemble trends
        ensemble_trends = ica.ensemble_trends_linear_regression(act, x_bin_size = x_bin_size, analysis_type = analysis_type, alpha_old = alpha_old, correction = correction)
        session_trends[session] = ensemble_trends
        ## Calculate proportions
        session_proportions[session] = ica.calculate_proportions_ensembles(ensemble_trends, function = 'linear_regression')
    mouse_ensemble_trends[mouse] = session_trends
    mouse_ensemble_proportions[mouse] = session_proportions

100%|██████████| 7/7 [01:00<00:00,  8.62s/it]


In [66]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_ensemble_proportions:
    prop_decreasing_T4.append(mouse_ensemble_proportions[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_ensemble_proportions[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'Mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'grey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'Mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Time',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image(pjoin(figure_path, 'proportion_fading_trials.png'))